### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action = "ignore", category = FutureWarning)


### Import Data and perform Data Pre-processing

In [3]:
mf = pd.read_csv('match.csv')
df = pd.read_csv('delivery.csv')

mf.replace(['Mumbai Indians','Kolkata Knight Riders','Royal Challengers Bangalore','Deccan Chargers','Chennai Super Kings','Rajasthan Royals','Delhi Daredevils','Gujarat Lions','Kings XI Punjab','Sunrisers Hyderabad','Rising Pune Supergiants','Kochi Tuskers Kerala','Pune Warriors','Rising Pune Supergiant'],['MI','KKR','RCB','DC','CSK','RR','DD','GL','KXIP','SRH','RPS','KTK','PW','RPS'],inplace=True)
df.replace(['Mumbai Indians','Kolkata Knight Riders','Royal Challengers Bangalore','Deccan Chargers','Chennai Super Kings','Rajasthan Royals','Delhi Daredevils','Gujarat Lions','Kings XI Punjab','Sunrisers Hyderabad','Rising Pune Supergiants','Kochi Tuskers Kerala','Pune Warriors','Rising Pune Supergiant'],['MI','KKR','RCB','DC','CSK','RR','DD','GL','KXIP','SRH','RPS','KTK','PW','RPS'],inplace=True)

### Creating new dataframes

In [4]:
name = df['bowler'].unique()
bowler = pd.DataFrame(data=name, columns=['name'],index=name)

matches = df.groupby(['bowler'])['match_id'].unique()
bowler['matches'] = matches.apply(lambda x: len(x))
bowler['runs'] = 0
df['total_runs'] = df['batsman_runs'] + df['extra_runs']
bowler['runs']= df.groupby(['bowler'])['total_runs'].sum()
bowler['balls'] = df.groupby(['bowler'])['ball'].count()

df['player_dismissed'].fillna(value=0)
boo = df['player_dismissed'].notnull()
df['wicket'] = boo*1
bowler['wicket'] = df.groupby(['bowler'])['wicket'].sum()

bowler['average'] = bowler['runs']/bowler['wicket']
bowler['average'] = bowler['average'].apply(lambda x: round(x,2))

bowler['overs'] = (bowler['balls'] // 6) + ((bowler['balls']%6)/10).apply(lambda x: round(x,2))

over_convert = ((bowler['balls']//6) + ((bowler['balls']%6) / 6 ))
bowler['economy'] = (bowler['runs'] / over_convert).apply(lambda x: round(x,2))

bowler['strike_rate'] = (bowler['balls'] / bowler['wicket']).apply(lambda x: round(x,2))
bowler.info()
bowler.head()

<class 'pandas.core.frame.DataFrame'>
Index: 355 entries, Tymal Mills to Kane Williamson
Data columns (total 9 columns):
name           355 non-null object
matches        355 non-null int64
runs           355 non-null int64
balls          355 non-null int64
wicket         355 non-null int32
average        355 non-null float64
overs          355 non-null float64
economy        355 non-null float64
strike_rate    355 non-null float64
dtypes: float64(4), int32(1), int64(3), object(1)
memory usage: 26.3+ KB


,name,matches,runs,balls,wicket,average,overs,economy,strike_rate
Tymal Mills,Tymal Mills,5,157,111,5,31.40,18.3,8.49,22.20
Aniket Choudhary,Aniket Choudhary,5,144,108,5,28.80,18.0,8.00,21.60
Yuzvendra Chahal,Yuzvendra Chahal,55,1600,1219,72,22.22,203.1,7.88,16.93
Sreenath Aravind,Sreenath Aravind,38,1057,788,48,22.02,131.2,8.05,16.42
Shane Watson,Shane Watson,94,2490,1961,100,24.90,326.5,7.62,19.61


In [5]:
name = df['batsman'].unique()
batsman = pd.DataFrame(data=name, columns=['name'],index=name)
matches = df.groupby(['batsman'])['match_id'].unique()
batsman['innings'] = matches.apply(lambda x: len(x))
batsman['runs'] = 0
batsman['runs']=df.groupby(['batsman'])['batsman_runs'].sum()
batsman['balls_faced'] = df.groupby(['batsman'])['ball'].count()

batsman['out'] = df.groupby(['batsman'])['player_dismissed'].count().to_frame()
batsman['not_out'] = batsman['innings'] - batsman['out']
batsman['average'] = (batsman['runs'] / batsman['out']).apply(lambda x: round(x, 2))
batsman['per_inn_avg'] = (batsman['runs'] / batsman['innings']).apply(lambda x: round(x, 2))
batsman['strike_rate'] = (batsman['runs'] * 100 / batsman['balls_faced']).apply(lambda x: round(x, 2))

batsman['sixes']=df.groupby('batsman')['batsman_runs'].agg(lambda x: (x==6).sum())
batsman['fours']=df.groupby(['batsman'])['batsman_runs'].agg(lambda x: (x==4).sum())



batsman.info()
batsman.head()

<class 'pandas.core.frame.DataFrame'>
Index: 461 entries, David Warner to Eklavya Dwivedi
Data columns (total 11 columns):
name           461 non-null object
innings        461 non-null int64
runs           461 non-null int64
balls_faced    461 non-null int64
out            461 non-null int64
not_out        461 non-null int64
average        457 non-null float64
per_inn_avg    461 non-null float64
strike_rate    461 non-null float64
sixes          461 non-null int64
fours          461 non-null int64
dtypes: float64(3), int64(7), object(1)
memory usage: 43.2+ KB


,name,innings,runs,balls_faced,out,not_out,average,per_inn_avg,strike_rate,sixes,fours
David Warner,David Warner,114,4014,2902,102,12,39.35,35.21,138.32,160,401
Shikhar Dhawan,Shikhar Dhawan,126,3561,3005,108,18,32.97,28.26,118.50,71,401
Moises Henriques,Moises Henriques,50,969,769,35,15,27.69,19.38,126.01,27,87
Yuvraj Singh,Yuvraj Singh,116,2591,2050,100,16,25.91,22.34,126.39,141,205
Deepak Hooda,Deepak Hooda,30,373,277,23,7,16.22,12.43,134.66,20,20


In [6]:
bowler.to_csv('bowler.csv')
batsman.to_csv('batsman.csv')

### Data Massaging

In [7]:
pf = pd.read_csv('2018_manual.csv')
pf = pf[['team1_id','team2_id','toss_winner','toss_decision','winner','venue']]
pf.head()

,team1_id,team2_id,toss_winner,toss_decision,winner,venue
0,MI,CSK,CSK,field,CSK,"Wankhede Stadium, Mumbai, Maharashtra"
1,DD,KXI,KXI,field,KXI,"Feroz Shah Kotla Stadium, Delhi"
2,KKR,RCB,KKR,field,KKR,"Eden Garden Stadium, Kolkata, West Bengal"
3,SRH,RR,SRH,field,SRH,"Rajiv Gandhi International Stadium, Hyderabad"
4,CSK,KKR,CSK,field,CSK,"M.A.Chidambaram Stadium Chennai, Tamil Nadu"


In [8]:
mf = mf[['team1_id','team2_id','toss_winner','toss_decision','winner','venue']]
mf = mf.append(pf,ignore_index=True)
mf.info()
mf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 6 columns):
team1_id         696 non-null object
team2_id         696 non-null object
toss_winner      696 non-null object
toss_decision    696 non-null object
winner           693 non-null object
venue            696 non-null object
dtypes: object(6)
memory usage: 32.7+ KB


,team1_id,team2_id,toss_winner,toss_decision,winner,venue
0,KKR,RCB,RCB,field,KKR,"M.Chinnaswamy Stadium Bengaluru, Karnataka"
1,CSK,KXI,CSK,bat,CSK,I.S. Bindra Punjab Cricket Association Stadium...
2,RR,DD,RR,bat,DD,"Feroz Shah Kotla Stadium, Delhi"
3,MI,RCB,MI,bat,RCB,"Wankhede Stadium, Mumbai, Maharashtra"
4,DC,KKR,DC,bat,KKR,"Eden Garden Stadium, Kolkata, West Bengal"


In [9]:
mf = mf.replace(to_replace='DC',value='SRH')
mf['winner'].fillna('Draw', inplace=True)
mf.replace({'team1_id': {'MI':1,'KKR':2,'RCB':3,'CSK':5,'RR':6,'DD':7,'GL':8,'KXI':9,'SRH':10,'RPS':11,'KTK':12,'PW':13},
          'team2_id': {'MI':1,'KKR':2,'RCB':3,'CSK':5,'RR':6,'DD':7,'GL':8,'KXI':9,'SRH':10,'RPS':11,'KTK':12,'PW':13},
          'toss_winner': {'MI':1,'KKR':2,'RCB':3,'CSK':5,'RR':6,'DD':7,'GL':8,'KXI':9,'SRH':10,'RPS':11,'KTK':12,'PW':13},
          'winner': {'MI':1,'KKR':2,'RCB':3,'CSK':5,'RR':6,'DD':7,'GL':8,'KXI':9,'SRH':10,'RPS':11,'KTK':12,'PW':13,'Draw':14}}
, inplace=True)
mf.head()

,team1_id,team2_id,toss_winner,toss_decision,winner,venue
0,2,3,3,field,2,"M.Chinnaswamy Stadium Bengaluru, Karnataka"
1,5,9,5,bat,5,I.S. Bindra Punjab Cricket Association Stadium...
2,6,7,6,bat,7,"Feroz Shah Kotla Stadium, Delhi"
3,1,3,1,bat,3,"Wankhede Stadium, Mumbai, Maharashtra"
4,10,2,10,bat,2,"Eden Garden Stadium, Kolkata, West Bengal"


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
mf['toss_decision'] = le.fit_transform(mf['toss_decision'])
mf['venue'] = le.fit_transform(mf['venue'])
mf.info()
mf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 6 columns):
team1_id         696 non-null int64
team2_id         696 non-null int64
toss_winner      696 non-null int64
toss_decision    696 non-null int64
winner           696 non-null int64
venue            696 non-null int64
dtypes: int64(6)
memory usage: 32.7 KB


,team1_id,team2_id,toss_winner,toss_decision,winner,venue
0,2,3,3,1,2,16
1,5,9,5,0,5,12
2,6,7,6,0,7,8
3,1,3,1,0,3,33
4,10,2,10,0,2,7


### Final Dataset to be Used

In [11]:
mf.to_csv('mf.csv',index=False)